In [62]:
import os
import face_recognition
import cv2
import pandas as pd
import numpy as np
import json

# 1. Data preparation

### identity_CelebA dataframe
here we have only `filename` column and corresponding `identity`

In [5]:
# reading dataset 'identity_CelebA'
df_identity = pd.read_csv('dataset/identity_CelebA.txt', sep=' ', names=['filename', 'identity'])
df_identity.head(3)

,filename,identity
0,000001.jpg,2880
1,000002.jpg,2937
2,000003.jpg,8692


leaving only those identities, who have more than 7 photos

In [124]:
distribution = df_identity['identity'].value_counts()  # counting how many images there are for each identity
distr = distribution.loc[distribution > 4]             # leaving identities, who has 7+ images 
idx_list = distr.index                                 # getting indexes of the left identities
len(idx_list)

9343

### Creating the DataFrame,where each row is the identity index and the corresponding filenames

In [126]:
def create_identities_dict(row):
    if row['identity'] in identities_dict:
        identities_dict[row['identity']][0].append(row['filename'])
    else:
        identities_dict[row['identity']] = [[row['filename']]]

In [127]:
%%time
identities_dict = {}

for index, row in df_identity.iterrows():
    create_identities_dict(row)

# creating identities/filenames DataFrame
df_if = pd.DataFrame.from_dict(identities_dict, orient='index').reset_index().rename(columns={"index": 'identities', 0: 'filenames'})

CPU times: user 12.3 s, sys: 7.99 ms, total: 12.3 s
Wall time: 12.4 s


In [128]:
df_if = df_if.loc[df_if['identities'].isin(idx_list)]
print('DataFrame has ' + str(len(df_if)) + ' rows')
df_if.head(5)

DataFrame has 9343 rows


,identity,filenames
0,2880,"[000001.jpg, 000404.jpg, 003415.jpg, 004390.jp..."
1,2937,"[000002.jpg, 011437.jpg, 016335.jpg, 017121.jp..."
2,8692,"[000003.jpg, 015648.jpg, 033840.jpg, 038887.jp..."
3,5805,"[000004.jpg, 001778.jpg, 010191.jpg, 013676.jp..."
4,9295,"[000005.jpg, 008431.jpg, 014427.jpg, 016680.jp..."


# 2. Creating data structures for face recognition algorithm
We need:  
`df_final` - a DataFrame where each identity has 5 face encodings  
`encodings` - array of all encodings  
`users` - dictionary, where keys are users' idx and the values are the corresponding names/addresses (here these are identities' idx, but in my project it will change)  
`df_test_images` - DataFrame with the images that weren't parsed into encodings earlier  
`df_test_encodings` - DataFrame with identities and images parsed into encodings  

### 2.1 `df_final`:

In [130]:
def image_to_encodings(filename):
    """
        - gets filename, reads the image, finds faces, creates face encodings
        - returns face encoding, if a face was found on the image
    """
    img = cv2.imread('dataset/img_align_celeba/' + filename)
    locations = face_recognition.face_locations(img, model='hog')
    encodings_ = face_recognition.face_encodings(img, locations)
    return encodings_

In [139]:
def find_new_image(identity, filenames_list):
    """
        - gets id of identity and list of filenames, which belongs to identity
        - proceeds a search of non-seen images and trying to recognize face on it
        - goes into recursion, if a face wasn't found
        - returns face encoding and filename of the image, on which the face was recognized
    """
    global seen_images
    for filename_ in filenames_list:
        if filename_ not in seen_images:
            encodings_ = image_to_encodings(filename_)
            seen_images.append(filename_)
            filenames_list.remove(filename_)
            if len(encodings_) != 1:
                new_encoding_, new_filename_ = find_new_image(identity, filenames_list)
                seen_images.append(new_filename_)
                return new_encoding_, new_filename_
            return encodings_[0], filename_
    return None, None
    

In [326]:
def filenames_into_encodings(df: pd.DataFrame):
    """
        - Takes a DataFrame with columns 'filenames' and 'identities'
        - Reads each image, creates face encodings, tries to find new image in case the face wasn't found 
        or skips damaged images/identities
        - Returns:
            - `encodings_overall` - list with all encodings
            - `errors` - log of images, on which there was 0 or more than 1 faces
            - `seen_images_` - list of seen images
            - `data_` - dictionary, where keys are identities idx and values are list of 5 face encodings
    """
    encodings_overall = []  # list of all encodings 
    errors_ = []  # list for images, where weren't located any faces
    seen_images_ = []
    data_ = {'identities': [], 'encodings': []}

    # for identity in df['identity'].iloc[:100].values:
    # be careful with the activation of the line below, it takes almost 2 hours to proceed
    for identity in df['identities'].values:
#         filenames_list = df['filenames'].loc[df['identities'] == identity].values[0]
        filenames_list = json.loads(df['filenames'].loc[df['identities'] == identity].values[0])
        counter = 0
        data_['identities'].append(identity)
        temp_encodings = []
        for filename in filenames_list:
            if counter >= 5:
                continue

            encodings_ = image_to_encodings(filename)
            seen_images_.append(filename)

            # if a face wasn't found or there were more than 1 face
            if len(encodings_) != 1:
                # try to find another image of the same identity
                new_encoding, new_filename = find_new_image(identity, filenames_list)

                # if there wasn't any other photos
                if new_encoding is None:
                    # append an error message
                    errors_.append(('problem in:', filename, new_filename, identity, 'solved with:', 'ISN\'T SOLVED'))
                else:
                    encodings_overall.append(new_encoding)
                    temp_encodings.append(list(new_encoding))
                    errors_.append(('problem in:', filename, identity, 'solved with:', new_filename))
            else:
                encodings_overall.append(encodings_[0])
                temp_encodings.append(list(encodings_[0]))

            counter += 1
        data_['encodings'].append(json.dumps(temp_encodings))
    return encodings_overall, errors_, seen_images_, data_

In [140]:
%%time
encodings, errors, seen_images, data = filenames_into_encodings(df_if)

CPU times: user 2h 28min, sys: 7.35 s, total: 2h 28min 8s
Wall time: 2h 35min 39s


In [308]:
len(seen_images)

48567

Creating and saving the dataframe

In [141]:
df_fin = pd.DataFrame(data=data, columns=['identities', 'encodings'])
df_fin.to_csv('identities_encodings_super_final.csv', index=False)
df_fin.head(5)

,identities,encodings
0,2880,"[[-0.09392920881509781, 0.16680526733398438, 0..."
1,2937,"[[-0.13565079867839813, 0.04019104316830635, 0..."
2,8692,"[[-0.08772975206375122, 0.17790649831295013, 0..."
3,5805,"[[-0.207429900765419, 0.17357361316680908, -0...."
4,9295,"[[-0.11555222421884537, 0.05275091528892517, 0..."


Reading the DF

In [142]:
df_read_fin = pd.read_csv('identities_encodings_super_final.csv')
df_read_fin

,identities,encodings
0,2880,"[[-0.09392920881509781, 0.16680526733398438, 0..."
1,2937,"[[-0.13565079867839813, 0.04019104316830635, 0..."
2,8692,"[[-0.08772975206375122, 0.17790649831295013, 0..."
3,5805,"[[-0.207429900765419, 0.17357361316680908, -0...."
4,9295,"[[-0.11555222421884537, 0.05275091528892517, 0..."
...,...,...
9338,5920,"[[-0.20153068006038666, 0.13730086386203766, 0..."
9339,9268,"[[-0.10642589628696442, 0.13724175095558167, 0..."
9340,8052,"[[0.06224041059613228, 0.005616001784801483, -..."
9341,8146,"[[-0.12671472132205963, 0.1192317008972168, 0...."


Checking how many face encodings each person has

In [143]:
def check_enc_quant(row):
    if len(json.loads(row['encodings'])) != 5:
        damaged_rows.append(row['identities'])

Finding the damaged rows

In [162]:
damaged_rows = []

for index, row in df_read_fin.iterrows():
    check_enc_quant(row)
    
len(damaged_rows)

60

Dropping the damaged rows and saving the result

In [163]:
df_read_fin = df_read_fin.loc[~df_read_fin['identities'].isin(damaged_rows)]
df_read_fin.to_csv('identities_encodings_super_final2.csv', index=False)
len(df_read_fin)

9283

### 2.2 `encodings`:

Reading the DataFrame

In [171]:
df_final = pd.read_csv('identities_encodings_super_final2.csv')
print(f'{len(df_final)} unique identities in the DataFrame')

9283 unique identities in the DataFrame


Creating list of all encodings

In [170]:
encodings = []
for index, row in df_final.iterrows():
    encodings.extend(json.loads(row['encodings']))

print(f'{len(encodings)} face encodings in the encodings list')

46415 face encodings in the encodings list


### 2.3 `users`:  
Creating map:  
- from encoding index // 5   
- to identity or user's name/address  

In [341]:
user_idx = df_final['identities']
users = {key: value for key, value in zip(range(len(user_idx)), user_idx)}

### 2.4 `df_test_images`:

Creating new DataFrame, which consists of images, which weren't seen during creation of the main dataset (`df_final`)

In [240]:
%%time
data_test = {'identities': [], 'filenames': []}

for index, row in df_if.iterrows():
    data_test['identities'].append(row['identity'])
    temp_list = []
    for filename in row['filenames']:
        if filename not in seen_images:
            temp_list.append(filename)
    data_test['filenames'].append(json.dumps(temp_list))

len(data_test['filenames'])

CPU times: user 3min 39s, sys: 19.9 ms, total: 3min 39s
Wall time: 3min 39s


9343

In [241]:
df_test = pd.DataFrame(data=data_test, columns=['identities', 'filenames'])
len(df_test)

9343

Dropping identities, who doesn't have any images

In [242]:
rows_to_drop = []

for index, row in df_test.iterrows():
    if len(row['filenames']) < 2:
        rows_to_drop.append(row['identities'])

len(rows_to_drop)

0

In [243]:
df_test_images = df_test.loc[~df_test['identities'].isin(rows_to_drop)]
df_test_images.to_csv('test_identities_filenames.csv', index=False)
len(df_test_images)

9343

### 2.5 `df_test_encodings`:

Creating DataFrame, where each identity will have his list of face encodings

In [321]:
%%time
test_encodings, test_errors, test_seen_images, test_data = filenames_into_encodings(df_test)

CPU times: user 2h 18min 34s, sys: 8.02 s, total: 2h 18min 42s
Wall time: 2h 34min 19s


Saving the DataFrame

In [322]:
df_test_encodings = pd.DataFrame(data=test_data, columns=['identities', 'encodings'])
df_test_encodings.to_csv('identities_encodings_test.csv', index=False)
df_test_encodings.head(5)

,identities,encodings
0,2880,"[[-0.08185373246669769, 0.10833615064620972, -..."
1,2937,"[[-0.08213772624731064, 0.0223865807056427, 0...."
2,8692,"[[-0.03371645510196686, 0.10744943469762802, 0..."
3,5805,"[[-0.2582927942276001, 0.08399071544408798, 0...."
4,9295,"[[-0.10332327336072922, 0.07786936312913895, 0..."


Reading the DataFrame

In [330]:
df_test_read = pd.read_csv('identities_encodings_test.csv')
df_test_read

,identities,encodings
0,2880,"[[-0.08185373246669769, 0.10833615064620972, -..."
1,2937,"[[-0.08213772624731064, 0.0223865807056427, 0...."
2,8692,"[[-0.03371645510196686, 0.10744943469762802, 0..."
3,5805,"[[-0.2582927942276001, 0.08399071544408798, 0...."
4,9295,"[[-0.10332327336072922, 0.07786936312913895, 0..."
...,...,...
9338,5920,"[[-0.18387816846370697, 0.0963742733001709, 0...."
9339,9268,"[[-0.21787138283252716, 0.15073078870773315, 0..."
9340,8052,[]
9341,8146,"[[-0.03438824787735939, 0.09196805953979492, 0..."


In [333]:
def find_identities_with_less_than_n_encodings(df: pd.DataFrame, n: int):
    identities_to_drop = []
    for index, row in df.iterrows():
        encodings_list = json.loads(row['encodings'])
        if len(encodings_list) < n:
            identities_to_drop.append(row['identities'])
    return identities_to_drop


def create_column_with_n_encodings(df: pd.DataFrame, n: int):
    n_encodings = []
    for index, row in df.iterrows():
        encodings_list = json.loads(row['encodings'])
        temp_encodings = []
        counter = 0
        for enc in encodings_list:
            if counter < n:
                temp_encodings.append(enc)
                counter += 1
        n_encodings.append(temp_encodings)
    return n_encodings

Cleaning identities, who has less than 5 encodings

In [334]:
idx_to_drop = find_identities_with_less_than_n_encodings(df_test_read, 5)
df_test_read = df_test_read.loc[~df_test_read['identities'].isin(idx_to_drop)]
len(df_test_read)

8270

Creating new column, where is left only 5 encodings for each identity

In [336]:
new_column = create_column_with_n_encodings(df_test_read, 5)
df_test_read.loc[:, ('5_encodings')] = new_column
df_test_read.head(3)

/tmp/ipykernel_4710/3922244095.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test_read.loc[:, ('5_encodings')] = new_column


,identities,encodings,5_encodings
0,2880,"[[-0.08185373246669769, 0.10833615064620972, -...","[[-0.08185373246669769, 0.10833615064620972, -..."
1,2937,"[[-0.08213772624731064, 0.0223865807056427, 0....","[[-0.08213772624731064, 0.0223865807056427, 0...."
2,8692,"[[-0.03371645510196686, 0.10744943469762802, 0...","[[-0.03371645510196686, 0.10744943469762802, 0..."


# 3. Testing perfomance of the different approaches  
## 3.1 Baseline  
- Take the first encoding of each person
- Choosing the best tolerance threshold
- Maximize the accuracy metric

In [417]:
np.arange(0.55, 0.66, 0.05)

array([0.55, 0.6 , 0.65])

In [503]:
# BASE CODE
def match_the_face(filename: str, encodings_list: list) -> np.array:
    """
        - Takes an image
        - Iterates through 0.2-0.8 values, which are TOLERANCE values
        - Returns a list with 7 predictions, some of them may be None
    """
    image = cv2.imread('dataset/img_align_celeba/' + filename)
    locations_ = face_recognition.face_locations(image, model='hog')
    encodings_ = face_recognition.face_encodings(image, locations_)

    if len(encodings_) != 1:
        print('wtf', len(encodings_))
        return None
        
    threshold_choosing = []
    
    for i in np.arange(0.5, 0.71, 0.1):
        results = face_recognition.compare_faces(encodings_list, encodings_[0], tolerance=i)
        if True in results:
            matched_identity_idx = results.index(True) // 5
            identity_actual = users[matched_identity_idx]
            threshold_choosing.append(identity_actual)
        else:
            threshold_choosing.append(None)
                
    return threshold_choosing


def match_encoding(encodings_list, encoding, np_ar):
    threshold_results = []
    
    for i in np.arange(np_ar[0], np_ar[1], np_ar[2]):
        encoding = np.array(encoding)
        results = face_recognition.compare_faces(encodings_list, encoding, tolerance=i)
        if True in results:
            # this line if we pass regular `encodings` list
#             matched_identity_idx = results.index(True) // 5
            # this line if we pass `encodings_1_per_identity` list
            matched_identity_idx = results.index(True)
            identity_actual = users[matched_identity_idx]
            threshold_results.append(identity_actual)
        else:
            threshold_results.append(None)
                
    return threshold_results 


def match_encoding_5inarow(encodings_list, encoding, np_ar):
    threshold_results = []
    
    for i in np.arange(np_ar[0], np_ar[1], np_ar[2]):
        encoding = np.array(encoding)
        results = face_recognition.compare_faces(encodings_list, encoding, tolerance=i)
        if True in results:
            # this line if we pass regular `encodings` list
#             matched_identity_idx = results.index(True) // 5
            # this line if we pass `encodings_1_per_identity` list
    
    
            matched_identity_idx = results.index(True)
            identity_actual = users[matched_identity_idx]
            threshold_results.append(identity_actual)
        else:
            threshold_results.append(None)
                
    return threshold_results


def calc_acc(array: list, true_value: int):
    """
        - Takes array of predictions and a `true_value`
        - Compares and counts accuracy
        - Returns accuracy for a list
    """
    right_preds = 0
    for el in array:
        if el == true_value:
            right_preds += 1
    accuracy = right_preds / len(array)
#     print('right_preds: ', right_preds, ' len of array ', len(array))
    return accuracy


def get_predictions(row, encodings_list, np_ar, limit: int = 5) -> list:
    """
        - Takes a row of `df_test_encodings` DataFrame
        - Goes into a for loop for each encoding and matches it to `encodings` list
        - Calls `match_encoding` function
    """
    predictions_for_each_image = []    
    for encoding in row['5_encodings']:
#         print(f'get_predictions(): Encoding: {encoding}')
        results = match_encoding(encodings_list, encoding, np_ar)        
        if results is None:
            continue        
        predictions_for_each_image.append(results)  
    return predictions_for_each_image


def get_results_by_threshold(predictions):
    """
        - Takes predictions from `get_predictions`
        - Parses the list into a dict
        - Returns the dict, where keys are threshold values and values are the presictions of identity
    """
    results_by_threshold = {0.55: [], 0.6: [], 0.65: []}
    for result in predictions:
        results_by_threshold[0.55].append(result[0])
        results_by_threshold[0.6].append(result[1])
        results_by_threshold[0.65].append(result[2])
    return results_by_threshold

        
def get_accuracy_by_threshold(row, results_by_threshold):
    """
        - Takes the dict from `get_results_by_threshold`
        - Calculates accuracy for each threshold
        - Return accuracies list
    """
    acc_by_threshold = []
    for key in results_by_threshold:
#         print(results_by_threshold[key], row['identities'])
        acc = calc_acc(results_by_threshold[key], row['identities'])
        acc_by_threshold.append(acc)  
    return acc_by_threshold


def get_accuracy(row, predictions, np_ar):
    """
        - Takes the dict from `get_results_by_threshold`
        - Calculates accuracy for each threshold
        - Return accuracies list
    """
    acc_by_threshold = []
    length = len(np.arange(np_ar[0], np_ar[1], np_ar[2]))
    for i in range(length):
        acc = calc_acc(predictions[:, i], row['identities'])
        acc_by_threshold.append(acc)  
    return acc_by_threshold


def create_accuracy_column(df, encodings_list, np_ar):
    """
        - Takes `df_test` DataFrame
        - Calls `get_predictions`, `get_results_by_threshold`, `get_accuracy_by_threshold`
        - Returns list of lists (for each person) with accuracies (for each tolerance threshold)
    """
    accuracy_list = [] 
#     for index, row in df.iloc[8000:8200].iterrows():
    for index, row in df.iterrows():
        predictions_for_each_image = get_predictions(row, encodings_list, np_ar)        
        predictions = np.array(predictions_for_each_image)
        
        acc_by_threshold = get_accuracy(row, predictions, np_ar)        
        accuracy_list.append(acc_by_threshold)
        
    return accuracy_list


def print_acc_by_threshold(accuracies, np_ar):
    accuracies = np.array(accuracies)
    
    for i, threshold in enumerate(np.arange(np_ar[0], np_ar[1], np_ar[2])):
        print(f'Accuracy with {threshold:.3f} threshold: {accuracies[:, i].mean():.2f}')

In [400]:
%%time
encodings_1_per_person = encodings[::5]
identities_accuracies = create_accuracy_column(df_test_read, encodings_1_per_person, np_ar=(0.5, 0.71, 0.1))

CPU times: user 4min 45s, sys: 10.7 s, total: 4min 56s
Wall time: 4min 56s


In [448]:
print_acc_by_threshold(identities_accuracies, (0.5, 0.71, 0.1))

Accuracy with 0.500 threshold: 0.62
Accuracy with 0.600 threshold: 0.69
Accuracy with 0.700 threshold: 0.11


In [430]:
%%time
identities_accuracies_0_55_0_65 = create_accuracy_column(df_test_read, encodings_1_per_person, np_ar=(0.55, 0.66, 0.05))

CPU times: user 4min 43s, sys: 10.9 s, total: 4min 54s
Wall time: 4min 54s


In [449]:
print_acc_by_threshold(identities_accuracies_0_55_0_65, (0.55, 0.66, 0.05))

Accuracy with 0.550 threshold: 0.77
Accuracy with 0.600 threshold: 0.69
Accuracy with 0.650 threshold: 0.34


In [445]:
%%time
identities_accuracies_0_525_0_575 = create_accuracy_column(df_test_read, encodings_1_per_person, np_ar=(0.525, 0.576, 0.025))

CPU times: user 4min 47s, sys: 10.9 s, total: 4min 58s
Wall time: 4min 58s


In [450]:
print_acc_by_threshold(identities_accuracies_0_525_0_575, (0.525, 0.576, 0.025))

Accuracy with 0.525 threshold: 0.72
Accuracy with 0.550 threshold: 0.77
Accuracy with 0.575 threshold: 0.76


## 3.2 Mean Encoding  
- Create one mean encoding from five given encodings for each person
- Choose the best tolerance threshold
- Maximize the accuracy metric

In [464]:
counter = 0
temp_encodings = []
mean_encodings = []
for enc in encodings:
    if counter < 5:
        temp_encodings.append(enc)
        counter += 1
        if counter == 5:
            mean_enc = np.mean(temp_encodings, axis=0)
            mean_encodings.append(mean_enc)
            counter = 0
            temp_encodings = []

len(mean_encodings)

9283

In [467]:
%%time
identities_accuracies_mean_enc = create_accuracy_column(df_test_read, mean_encodings, np_ar=(0.5, 0.71, 0.1))

CPU times: user 29.6 s, sys: 10.3 s, total: 40 s
Wall time: 40 s


In [468]:
print_acc_by_threshold(identities_accuracies_mean_enc, (0.5, 0.71, 0.1))

Accuracy with 0.500 threshold: 0.89
Accuracy with 0.600 threshold: 0.22
Accuracy with 0.700 threshold: 0.02


In [472]:
%%time
identities_accuracies_mean_enc_0_45_0_55 = create_accuracy_column(df_test_read, mean_encodings, np_ar=(0.45, 0.551, 0.05))

CPU times: user 30.9 s, sys: 9.81 s, total: 40.7 s
Wall time: 40.7 s


In [473]:
print_acc_by_threshold(identities_accuracies_mean_enc_0_45_0_55, (0.45, 0.551, 0.05))

Accuracy with 0.450 threshold: 0.86
Accuracy with 0.500 threshold: 0.89
Accuracy with 0.550 threshold: 0.67


In [478]:
%%time
identities_accuracies_mean_enc_0_475_0_525 = create_accuracy_column(df_test_read, mean_encodings, np_ar=(0.475, 0.526, 0.025))

CPU times: user 53.2 s, sys: 10.2 s, total: 1min 3s
Wall time: 1min 3s


In [479]:
print_acc_by_threshold(identities_accuracies_mean_enc_0_475_0_525, (0.475, 0.526, 0.025))

Accuracy with 0.475 threshold: 0.49
Accuracy with 0.500 threshold: 0.22
Accuracy with 0.525 threshold: 0.06


In [480]:
%%time
identities_accuracies_mean_enc_0_463_0_487 = create_accuracy_column(df_test_read, mean_encodings, np_ar=(0.463, 0.488, 0.012))

CPU times: user 1min 6s, sys: 10.6 s, total: 1min 17s
Wall time: 1min 17s


In [481]:
print_acc_by_threshold(identities_accuracies_mean_enc_0_463_0_487, (0.463, 0.488, 0.012))

Accuracy with 0.463 threshold: 0.59
Accuracy with 0.475 threshold: 0.49
Accuracy with 0.487 threshold: 0.35


In [482]:
%%time
identities_accuracies_mean_enc_0_439_0_463 = create_accuracy_column(df_test_read, mean_encodings, np_ar=(0.439, 0.464, 0.012))

CPU times: user 1min 15s, sys: 10.2 s, total: 1min 25s
Wall time: 1min 25s


In [483]:
print_acc_by_threshold(identities_accuracies_mean_enc_0_439_0_463, (0.439, 0.464, 0.012))

Accuracy with 0.439 threshold: 0.71
Accuracy with 0.451 threshold: 0.65
Accuracy with 0.463 threshold: 0.59


In [484]:
%%time
identities_accuracies_mean_enc_0_415_0_44 = create_accuracy_column(df_test_read, mean_encodings, np_ar=(0.415, 0.44, 0.012))

CPU times: user 1min 18s, sys: 10.5 s, total: 1min 29s
Wall time: 1min 29s


In [485]:
print_acc_by_threshold(identities_accuracies_mean_enc_0_415_0_44, (0.415, 0.44, 0.012))

Accuracy with 0.415 threshold: 0.70
Accuracy with 0.427 threshold: 0.72
Accuracy with 0.439 threshold: 0.71


#### Test on the entire dataset:

In [508]:
%%time
identities_accuracies_mean_enc_0_3_0_5 = create_accuracy_column(df_test_read, mean_encodings, np_ar=(0.3, 0.501, 0.01))

CPU times: user 4h 30min 16s, sys: 49min 22s, total: 5h 19min 39s
Wall time: 5h 19min 43s


In [509]:
print_acc_by_threshold(identities_accuracies_mean_enc_0_3_0_5, (0.3, 0.501, 0.01))

Accuracy with 0.300 threshold: 0.15
Accuracy with 0.310 threshold: 0.19
Accuracy with 0.320 threshold: 0.24
Accuracy with 0.330 threshold: 0.29
Accuracy with 0.340 threshold: 0.34
Accuracy with 0.350 threshold: 0.40
Accuracy with 0.360 threshold: 0.46
Accuracy with 0.370 threshold: 0.51
Accuracy with 0.380 threshold: 0.56
Accuracy with 0.390 threshold: 0.61
Accuracy with 0.400 threshold: 0.65
Accuracy with 0.410 threshold: 0.69
Accuracy with 0.420 threshold: 0.72
Accuracy with 0.430 threshold: 0.74
Accuracy with 0.440 threshold: 0.74
Accuracy with 0.450 threshold: 0.73
Accuracy with 0.460 threshold: 0.71
Accuracy with 0.470 threshold: 0.65
Accuracy with 0.480 threshold: 0.58
Accuracy with 0.490 threshold: 0.49
Accuracy with 0.500 threshold: 0.39


## 3.3 Minimum accordance amount
- After getting a result from `compare_faces` function, look for 3, 4, 5 True values in a row and return the prediction  
- Choose the best `min_accordance` value (3-5)  
- Choose the best tolerance threshold  
- Maximize the accuracy metric  

## 3.4 Face distance  
- Use `face_distance` instead of compare faces  
- Calculate overall distance by 5 encodings for each person 
- Choosing the lowest value and the corresponding person

Сделать find_new_img() на случай, если на картинке не распозналось лицо. Проверить эту функцию на 14 примерах сверху. Если все ок - прогоняю все 46715 картинок в енкодинги лиц, получаю большой питоновский список. Можно сформировать новую колонку в датафрейме, для удобства. 
(СДЕЛАНО)

Описываю несколько вариантов алгоритма, сравнивающего лицо с пятью сохраненными енкодингами. Сначала пробуем менять порог, используя только первый енкодинг. Дальше можно сделать среднее из 5 енкодингов. Замерять среднюю разницу к каждому из 5 енкодингов, попробовать настроить это как второй порог, перебирая с основным.

# БУКУК

In [125]:
df = df_identity.loc[df_identity['identity'].isin(idx_list)]  # leaving only needed identities
df.head()

,filename,identity
0,000001.jpg,2880
1,000002.jpg,2937
2,000003.jpg,8692
3,000004.jpg,5805
4,000005.jpg,9295


leaving only 5 images in DataFrame for each identity

In [232]:
%%time
# leave only 5 rows for each identity
filenames_fin = []
idx_map = {}
for index, row in df.iterrows():
    if row['identity'] in idx_map.keys():
        if idx_map[row['identity']] < 5:
            filenames_fin.append(row['filename'])
            idx_map[row['identity']] += 1
    else:
        filenames_fin.append(row['filename'])
        idx_map[row['identity']] = 1

# leaving images only with the names, which we got on the previous step
df5 = df.loc[df['filename'].isin(filenames_fin)]
df5.head(3)

CPU times: user 7.12 s, sys: 3.76 ms, total: 7.12 s
Wall time: 7.12 s


,filename,identity
0,000001.jpg,2880
1,000002.jpg,2937
2,000003.jpg,8692


checking how many identities and images do we left with

In [233]:
print(f'We have {df5["identity"].value_counts().size} identities'
      f'\nEach identity has {df5["identity"].value_counts().value_counts().index[0]} images')

We have 9343 identities
Each identity has 5 images


In [32]:
df_fin = pd.DataFrame(data=data, columns=['identities', 'encodings'])
df_fin.to_csv('identities_encodings.csv', index=False)
df_fin.head(5)

,identities,encodings
0,2880,"[[-0.09392920881509781, 0.16680526733398438, 0..."
1,2937,"[[-0.13565079867839813, 0.04019104316830635, 0..."
2,8692,"[[-0.0700308233499527, 0.09956848621368408, 0...."
3,5805,"[[-0.26122236251831055, 0.11445462703704834, 0..."
4,9295,"[[-0.11555222421884537, 0.05275091528892517, 0..."


In [33]:
read_df = pd.read_csv('identities_encodings.csv')
read_df

,identities,encodings
0,2880,"[[-0.09392920881509781, 0.16680526733398438, 0..."
1,2937,"[[-0.13565079867839813, 0.04019104316830635, 0..."
2,8692,"[[-0.0700308233499527, 0.09956848621368408, 0...."
3,5805,"[[-0.26122236251831055, 0.11445462703704834, 0..."
4,9295,"[[-0.11555222421884537, 0.05275091528892517, 0..."
...,...,...
8910,6223,"[[-0.05556920915842056, 0.12527385354042053, 0..."
8911,6258,"[[-0.007462053559720516, -0.006663414649665356..."
8912,10101,"[[-0.0996573269367218, 0.03893882781267166, 0...."
8913,5920,"[[-0.20153068006038666, 0.13730086386203766, 0..."


identities with proper amount of encodings

In [54]:
read_df.drop(read_df.loc[read_df['identities'].isin(damaged_rows)].index, inplace=True)
len(read_df)

7783

In [75]:
df_new = df_fin.loc[df_fin['identities'].isin(damaged_rows)]
df_new

,identities,encodings
2,8692,"[[-0.0700308233499527, 0.09956848621368408, 0...."
3,5805,"[[-0.26122236251831055, 0.11445462703704834, 0..."
7,6369,"[[0.03670291602611542, 0.17831860482692719, -0..."
16,667,"[[-0.14975506067276, 0.09124454110860825, 0.14..."
30,268,"[[-0.11092862486839294, 0.13797610998153687, 0..."
...,...,...
8871,8214,"[[-0.07532814890146255, 0.1283508688211441, 0...."
8879,9549,"[[-0.06116217374801636, 0.057062678039073944, ..."
8895,9229,"[[-0.17309236526489258, 0.03611578419804573, 0..."
8899,6245,"[[-0.10766778141260147, 0.07498753815889359, 0..."


In [106]:
%%time
# creating list of all encodings 
encodings_new = []
# list for images, where weren't located any faces
errors_new = []
seen_images = []
data_new = {'identities': [], 'encodings': []}

# for identity in df_new['identities'].iloc[:100].values:
# be careful with the activation of the line below, it takes almost 2 hours to proceed
for identity in df_new['identities'].values:
    filenames_list = df_if['filenames'].loc[df_if['identity'] == identity].values[0]
    counter = 0
    data_new['identities'].append(identity)
    temp_encodings = []
    for filename in filenames_list:
        if counter >= 5:
            continue
            
        encodings_ = image_to_encodings(filename)
        seen_images.append(filename)
        
        # if a face wasn't found or there were more than 1 face
        if len(encodings_) != 1:
            # try to find another image of the same identity
            new_encoding, new_filename = find_new_image(identity, filenames_list)
            
            # if there wasn't any other photos
            if new_encoding is None:
                # append an error message
                errors_new.append(('problem in:', filename, new_filename, identity, 'solved with:', 'ISN\'T SOLVED'))
            else:
                encodings_new.append(new_encoding)
                temp_encodings.append(list(new_encoding))
                errors_new.append(('problem in:', filename, identity, 'solved with:', new_filename))
        else:
            encodings_new.append(encodings_[0])
            temp_encodings.append(list(encodings_[0]))
        
        counter += 1
    data_new['encodings'].append(json.dumps(temp_encodings))

CPU times: user 19min 1s, sys: 1.11 s, total: 19min 3s
Wall time: 20min 25s


Saving the results

In [119]:
df_fin2 = pd.DataFrame(data=data_new, columns=['identities', 'encodings'])
df_fin2.to_csv('identities_encodings_part2.csv', index=False)
len(df_fin2)

1132

Removing those, who had not enough images or the images were corrupted

In [112]:
drop_list = []
for error in errors_new:
    if error[-1] == "ISN'T SOLVED":
        drop_list.append(error[3])
drop_list

In [121]:
df_fin2 = df_fin2.loc[~df_fin2['identities'].isin(drop_list)]
df_fin2.to_csv('identities_encodings_part2.csv', index=False)
len(df_fin2)

1123

Checking the damaged rows

In [122]:
damaged_rows_new = []

for index, row in df_fin2.iterrows():
    check_enc_quant(row)

len(damaged_rows_new)

0

In [ ]:
df_fin_new = pd.DataFrame(data=data_new, columns=['identities', 'encodings'])
# df_fin_new.to_csv('identities_encodings.csv', index=False)
df_fin_new.head(5)

In [2]:
KNOWN_FACES_DIR = 'KNOWN_FACES'
UNKNOWN_FACES_DIR = 'UNKNOWN_FACES'
TOLERANCE = 0.6
FRAME_THIKNESS = 3
FONT_THIKNESS = 2
MODEL = 'hog'

# video = cv2.VideoCapture(0)

In [ ]:
known_faces = []
known_names = []

for name in os.listdir(KNOWN_FACES_DIR):
    for filename in os.listdir(f'{KNOWN_FACES_DIR}/{name}'):
        image = face_recognition.load_image_file(f'{KNOWN_FACES_DIR}/{name}/{filename}')
        encoding = face_recognition.face_encodings(image)
        known_faces.append(encoding[0])
        known_names.append(name)

In [ ]:
while True:
    # print(filename)
    # image = face_recognition.load_image_file(f'{UNKNOWN_FACES_DIR}/{filename}')

    ret, image = video.read()

    print(type(image))
    locations = face_recognition.face_locations(image, model=MODEL)
    encodings = face_recognition.face_encodings(image, locations)
    print(encodings)
    print(type(encodings))
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    for face_encoding, face_location in zip(encodings, locations):
        results = face_recognition.compare_faces(known_faces, face_encoding, TOLERANCE)
        match = None
        if True in results:
            match = known_names[results.index(True)]
            print(f'Match found: {match}')
            top_left = (face_location[3], face_location[0])
            bottom_right = (face_location[1], face_location[2])
            color = [0, 255, 0]
            cv2.rectangle(image, top_left, bottom_right, color, FRAME_THIKNESS)

            top_left = (face_location[3], face_location[2])
            bottom_right = (face_location[1], face_location[2] + 22)
            cv2.rectangle(image, top_left, bottom_right, color, cv2.FILLED)
            cv2.putText(image, match, (face_location[3] + 10, face_location[2] + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                        (200, 200, 200), FONT_THIKNESS)

    cv2.imshow('Webcam', image)
    if cv2.waitKey(20) & 0xFF==ord('d'):
        break
    # cv2.waitKey(0)
    # cv2.destroyWindow(filename)